In [1]:
import numpy as np
import random
%matplotlib notebook

import matplotlib.pyplot as plt
from scipy.stats import entropy
from sklearn.decomposition import PCA,  IncrementalPCA
import math
from sklearn import svm
from decimal import Decimal
import math
from sklearn import preprocessing
import heapq

In [2]:
def iauc(P,Q):
    return 1- np.sum(np.min(P,Q))/np.sum(P)
def kl(p, q):
    """Kullback-Leibler divergence D(P || Q) for discrete distributions
    Parameters
    ----------
    p, q : array-like, dtype=float, shape=n
    Discrete probability distributions.
    """
    epsilon = 0.00001
    p = p +epsilon 
    p = p/np.sum(p)
    q = q + epsilon
    q = q/np.sum(q)

    return entropy(p, q)
def mkl(p,q):
    return np.max([kl(p,q), kl(q,p)])
def kl2(P,Q):
    """ Epsilon is used here to avoid conditional code for checking that neither P nor Q is equal to 0. """
    epsilon = 0.00001
    
     # You may want to instead make copies to avoid changing the np arrays.
    P = P+epsilon
    P = P/np.sum(P)
    Q = Q+epsilon
    Q = Q/np.sum(Q)
    

    divergence = np.sum(P*np.log(P/Q))
    divergence = 0
    for i in range(len(P)):
        
        divergence += np.max([P[i] * math.log(P[i]/Q[i]), Q[i]* math.log(Q[i]/P[i])])
    return divergence

In [3]:
groud_truth = []
def generate_dataset(distribution_type, dim, length_chunk, mean, cov, change_value, change_type,change_param, num_chunk):
    global groud_truth
    out = []
    if distribution_type == "gauss":
        for i in range(num_chunk):
            generated_length_chunk = length_chunk
            
            #generated_length_chunk = length_chunk
            x = np.random.multivariate_normal(mean, cov, generated_length_chunk)

            if change_param == "mean":
                if change_type == "sudden":
                    for j  in range(len(mean)):
                        
                            mean[j] += change_value
                
                elif change_type == "incremental":
                    incremental_x = []
                    for t in range(5):
                        for j in range(len(mean)):
                            mean[j] += change_value/5
                        part_x = np.random.multivariate_normal(mean, cov, 1000)
                        for xi in part_x:
                            incremental_x.append(xi)
                elif change_type == "gradual":
                    gradual_x = []
                    new_mean = []
                    for j in range(len(mean)):
                        new_mean.append(mean[j] + change_value)
                    
                    for t in range(5):
                        threshold = (t+1)*10/5
                        r = random.randint(0,10)
                        if r < threshold:
                            part_x = np.random.multivariate_normal(new_mean, cov, 1000)
                        else:
                            part_x = np.random.multivariate_normal(mean, cov, 1000)
                        for xi in part_x:
                            gradual_x.append(xi)
                    mean = new_mean
                    
                    
                        
            elif change_param == "variance":
                if change_type == "sudden":
                    t = random.randint(0,2)
                    for j in range(len(cov[0])):
                        
                        if t% 2 == 0:
                            cov[j][j] = (math.sqrt(cov[j][j]) + change_value*math.sqrt(cov[j][j]))*(math.sqrt(cov[j][j]) + change_value*math.sqrt(cov[j][j]))
                        else:
                            cov[j][j] = (math.sqrt(cov[j][j]) - change_value*math.sqrt(cov[j][j]))*(math.sqrt(cov[j][j]) - change_value*math.sqrt(cov[j][j]))
                        for k in range(len(cov)):
                            if k != j and k < j:
                                cov[j][k] = 0.5 * cov[j][j]
                                cov[k][j] = cov[j][k]
                        
                    print(cov)
                elif change_type == "incremental":
                    incremental_x = []
                    t = random.randint(0,2)
                    for t_ in range(5):
                        for j in range(len(cov[0])):
                            
                            if t% 2 == 0:
                                cov[j][j] = (math.sqrt(cov[j][j]) + change_value*math.sqrt(cov[j][j])/5)*(math.sqrt(cov[j][j]) + change_value*math.sqrt(cov[j][j])/5)
                            else:
                                cov[j][j] = (math.sqrt(cov[j][j]) - change_value*math.sqrt(cov[j][j])/5)*(math.sqrt(cov[j][j]) - change_value*math.sqrt(cov[j][j])/5)
                            for k in range(len(cov)):
                                if k != j and k < j:
                                    cov[j][k] = 0.5 * cov[j][j]
                                    cov[k][j] = cov[j][k]
                        part_x = np.random.multivariate_normal(mean, cov, 1000)
                        for xi in part_x:
                            incremental_x.append(xi)
                        #print(cov)
                elif change_type == "gradual":
                    gradual_x = []
                    new_cov = cov
                    t = random.randint(0,2)
                    for j in range(len(new_cov[0])):
                        
                        if t% 2 == 0:
                            new_cov[j][j] = (math.sqrt(new_cov[j][j]) + change_value*math.sqrt(new_cov[j][j]))*(math.sqrt(new_cov[j][j]) + change_value*math.sqrt(new_cov[j][j]))
                        else:
                            new_cov[j][j] = (math.sqrt(new_cov[j][j]) - change_value*math.sqrt(new_cov[j][j]))*(math.sqrt(new_cov[j][j]) - change_value*math.sqrt(new_cov[j][j]))
                        for k in range(len(cov)):
                            if k != j and k < j:
                                new_cov[j][k] = 0.5 * new_cov[j][j]
                                new_cov[k][j] = new_cov[j][k]
                    for t_ in range(5):
                        threshold = (t_+1)/5*10
                        r = random.randint(0,10)
                        if r < threshold:
                            part_x = np.random.multivariate_normal(mean, new_cov, 1000)
                        else:
                            part_x = np.random.multivariate_normal(mean, cov, 1000)
                            
                        for xi in part_x:
                            gradual_x.append(xi)
                    #print(new_cov)
                    cov = new_cov
                                
                                
            elif change_param =="correlation":
                if change_type == "sudden":
                    for j in range(len(cov[0])):
                        for k in range(len(cov[0])):
                            if  j !=k and k < j:
                                if cov[j][k]+change_value * cov[j][j] < cov[j][j]:
                                    cov[j][k] += change_value * cov[j][j]
                                    cov[k][j] = cov[j][k]
                                else :
                                    cov[j][k] = 0.5 * cov[j][j]
                                    cov[k][j] = cov[j][k]
                    #print(cov)
            #x = x.reshape(int(len(x)/2),2)
                elif change_type == "incremental":
                    incremental_x = []
                    for t in range(5):
                        for j in range(len(cov[0])):
                            for k in range(len(cov[0])):
                                if  j !=k and k < j:
                                    if cov[j][k]+change_value/5 * cov[j][j] < cov[j][j]:
                                        cov[j][k] += change_value/5 * cov[j][j]
                                        cov[k][j] = cov[j][k]
                                    else :
                                        cov[j][k] = 0.5 * cov[j][j]
                                        cov[k][j] = cov[j][k]
                        #print(cov)
                        part_x = np.random.multivariate_normal(mean, cov, 1000)
                        for xi in part_x:
                            incremental_x.append(xi)
                elif change_type == "gradual":
                    gradual_x = []
                    new_cov = cov
                    for j in range(len(new_cov[0])):
                        for k in range(len(new_cov[0])):
                            if  j !=k and k < j:
                                if new_cov[j][k]+change_value * new_cov[j][j] < new_cov[j][j]:
                                    new_cov[j][k] += change_value * new_cov[j][j]
                                    new_cov[k][j] = new_cov[j][k]
                                else :
                                    new_cov[j][k] = 0.5 * new_cov[j][j]
                                    new_cov[k][j] = new_cov[j][k]
                    for t_ in range(5):
                        threshold = (t_+1)/5*10
                        r = random.randint(0,10)
                        if r < threshold:
                            part_x = np.random.multivariate_normal(mean, new_cov, 1000)
                        else:
                            part_x = np.random.multivariate_normal(mean, cov, 1000)
                            
                        for xi in part_x:
                            gradual_x.append(xi)
                    #print(new_cov)
                    cov = new_cov 
            if i > 0:
                groud_truth.append(len(out))
            if change_type == "sudden" or len(out) == 0:
                for xi in x:
                    out.append(xi)
            elif change_type == "incremental":
                for xi in x[5000:]:
                    out.append(xi)
                for xi in incremental_x:
                    out.append(xi)
            elif change_type == "gradual":
                for xi in x[5000:]:
                    out.append(xi)
                for xi in gradual_x:
                    out.append(xi)
            
    return np.array(out)

In [4]:
groud_truth = []
true_outliers = []
def generate_dataset_with_changes_and_outliers(distribution_type, dim, length_chunk, mean, cov, change_value, change_type,change_param, num_chunk, outlier_rate):
    global groud_truth
    global true_outliers
    out = []
    if distribution_type == "gauss":
        for i in range(num_chunk):
            generated_length_chunk = random.randint(length_chunk,length_chunk * 5)
            
            generated_length_chunk = length_chunk
            x = np.random.multivariate_normal(mean, cov, generated_length_chunk)

            if change_param == "mean":
                if change_type == "sudden":
                    for j  in range(len(mean)):
                        
                            mean[j] += change_value
                
                elif change_type == "incremental":
                    incremental_x = []
                    for t in range(5):
                        for j in range(len(mean)):
                            mean[j] += change_value/5
                        part_x = np.random.multivariate_normal(mean, cov, 1000)
                        for xi in part_x:
                            incremental_x.append(xi)
                elif change_type == "gradual":
                    gradual_x = []
                    new_mean = []
                    for j in range(len(mean)):
                        new_mean.append(mean[j] + change_value)
                    
                    for t in range(5):
                        threshold = (t+1)*10/5
                        r = random.randint(0,10)
                        if r < threshold:
                            part_x = np.random.multivariate_normal(new_mean, cov, 1000)
                        else:
                            part_x = np.random.multivariate_normal(mean, cov, 1000)
                        for xi in part_x:
                            gradual_x.append(xi)
                    mean = new_mean
                    
                    
                        
            elif change_param == "variance":
                if change_type == "sudden":
                    t = random.randint(0,2)
                    for j in range(len(cov[0])):
                        
                        if t% 2 == 0:
                            cov[j][j] = (math.sqrt(cov[j][j]) + change_value*math.sqrt(cov[j][j]))*(math.sqrt(cov[j][j]) + change_value*math.sqrt(cov[j][j]))
                        else:
                            cov[j][j] = (math.sqrt(cov[j][j]) - change_value*math.sqrt(cov[j][j]))*(math.sqrt(cov[j][j]) - change_value*math.sqrt(cov[j][j]))
                        for k in range(len(cov)):
                            if k != j and k < j:
                                cov[j][k] = 0.5 * cov[j][j]
                                cov[k][j] = cov[j][k]
                        
                    print(cov)
                elif change_type == "incremental":
                    incremental_x = []
                    t = random.randint(0,2)
                    for t_ in range(5):
                        for j in range(len(cov[0])):
                            
                            if t% 2 == 0:
                                cov[j][j] = (math.sqrt(cov[j][j]) + change_value*math.sqrt(cov[j][j])/5)*(math.sqrt(cov[j][j]) + change_value*math.sqrt(cov[j][j])/5)
                            else:
                                cov[j][j] = (math.sqrt(cov[j][j]) - change_value*math.sqrt(cov[j][j])/5)*(math.sqrt(cov[j][j]) - change_value*math.sqrt(cov[j][j])/5)
                            for k in range(len(cov)):
                                if k != j and k < j:
                                    cov[j][k] = 0.5 * cov[j][j]
                                    cov[k][j] = cov[j][k]
                        part_x = np.random.multivariate_normal(mean, cov, 1000)
                        for xi in part_x:
                            incremental_x.append(xi)
                        #print(cov)
                elif change_type == "gradual":
                    gradual_x = []
                    new_cov = cov
                    t = random.randint(0,2)
                    for j in range(len(new_cov[0])):
                        
                        if t% 2 == 0:
                            new_cov[j][j] = (math.sqrt(new_cov[j][j]) + change_value*math.sqrt(new_cov[j][j]))*(math.sqrt(new_cov[j][j]) + change_value*math.sqrt(new_cov[j][j]))
                        else:
                            new_cov[j][j] = (math.sqrt(new_cov[j][j]) - change_value*math.sqrt(new_cov[j][j]))*(math.sqrt(new_cov[j][j]) - change_value*math.sqrt(new_cov[j][j]))
                        for k in range(len(cov)):
                            if k != j and k < j:
                                new_cov[j][k] = 0.5 * new_cov[j][j]
                                new_cov[k][j] = new_cov[j][k]
                    for t_ in range(5):
                        threshold = (t_+1)/5*10
                        r = random.randint(0,10)
                        if r < threshold:
                            part_x = np.random.multivariate_normal(mean, new_cov, 1000)
                        else:
                            part_x = np.random.multivariate_normal(mean, cov, 1000)
                            
                        for xi in part_x:
                            gradual_x.append(xi)
                    #print(new_cov)
                    cov = new_cov
                                
                                
            elif change_param =="correlation":
                if change_type == "sudden":
                    for j in range(len(cov[0])):
                        for k in range(len(cov[0])):
                            if  j !=k and k < j:
                                if cov[j][k]+change_value * cov[j][j] < cov[j][j]:
                                    cov[j][k] += change_value * cov[j][j]
                                    cov[k][j] = cov[j][k]
                                else :
                                    cov[j][k] = 0.5 * cov[j][j]
                                    cov[k][j] = cov[j][k]
                    #print(cov)
            #x = x.reshape(int(len(x)/2),2)
                elif change_type == "incremental":
                    incremental_x = []
                    for t in range(5):
                        for j in range(len(cov[0])):
                            for k in range(len(cov[0])):
                                if  j !=k and k < j:
                                    if cov[j][k]+change_value/5 * cov[j][j] < cov[j][j]:
                                        cov[j][k] += change_value/5 * cov[j][j]
                                        cov[k][j] = cov[j][k]
                                    else :
                                        cov[j][k] = 0.5 * cov[j][j]
                                        cov[k][j] = cov[j][k]
                        #print(cov)
                        part_x = np.random.multivariate_normal(mean, cov, 1000)
                        for xi in part_x:
                            incremental_x.append(xi)
                elif change_type == "gradual":
                    gradual_x = []
                    new_cov = cov
                    for j in range(len(new_cov[0])):
                        for k in range(len(new_cov[0])):
                            if  j !=k and k < j:
                                if new_cov[j][k]+change_value * new_cov[j][j] < new_cov[j][j]:
                                    new_cov[j][k] += change_value * new_cov[j][j]
                                    new_cov[k][j] = new_cov[j][k]
                                else :
                                    new_cov[j][k] = 0.5 * new_cov[j][j]
                                    new_cov[k][j] = new_cov[j][k]
                    for t_ in range(5):
                        threshold = (t_+1)/5*10
                        r = random.randint(0,10)
                        if r < threshold:
                            part_x = np.random.multivariate_normal(mean, new_cov, 1000)
                        else:
                            part_x = np.random.multivariate_normal(mean, cov, 1000)
                            
                        for xi in part_x:
                            gradual_x.append(xi)
                    #print(new_cov)
                    cov = new_cov 
            if i > 0:
                groud_truth.append(len(out))
            if change_type == "sudden" or len(out) == 0:
                for xi in x:
                    out.append(xi)
            elif change_type == "incremental":
                for xi in x[5000:]:
                    out.append(xi)
                for xi in incremental_x:
                    out.append(xi)
            elif change_type == "gradual":
                for xi in x[5000:]:
                    out.append(xi)
                for xi in gradual_x:
                    out.append(xi)
            
    return np.array(out)

In [5]:
def change_detection_AVG(data, window_size, slide_size):
    global threshold_avg
    threshold = threshold_avg
    
    global alpha
    
    dim = len(data[0,:])
    #print("dim = "+ str(dim))
    start_idx = 0
    end_idx = window_size
    #get reference 
    reference_window = data[0:window_size,:]
    #print(reference_window[4999])
    ref_hists = []
    bin_edges = []
    num_selected_components = 0
    
    
    reference_window, bin_edges, dim, pca, ref_hists,num_selected_components = get_reference_window(reference_window, dim)
    #alpha = x1/(len(bin_edges)-1)
    #print("Num bin = " + str(len(bin_edges[0])))
    
    count_slide = 0
    start_idx += slide_size
    end_idx += slide_size
    ref_end_idx = end_idx
    distances = []
    changes = []
    
    current_avg = 0
    count_avg =0
    avg_Sc = 0
    while(end_idx < len(data)):
        count_slide +=1
        
        current_window = data[start_idx: end_idx]
        current_window = pca.transform(current_window)
        #get current hist
        current_hists = get_hists(current_window,num_selected_components, bin_edges)
        #distance = distance2(current_hists,ref_hists, pca)
        distance = distance_max2(ref_hists,current_hists, pca)
        #distance = distance_min(current_hists,ref_hists, pca)
        #distance = distance_sum(current_hists,ref_hists, pca)
        #distance = distance_sum_square(current_hists,ref_hists, pca)
        if count_slide %50000 == 0:
            #print("Count slide = "+ str(count_slide))
            #print("distance = "+ str(distance))
            pass
        if start_idx > ref_end_idx:
            distances.append(distance)
        
        
        
        
        if len(distances) > window_size/slide_size:
            del distances[0]
            current_avg = np.sum(distances)/len(distances)
            avg_Sc = (avg_Sc * count_avg + current_avg) /(count_avg +1)
            count_avg +=1
        #print(distance)
        if start_idx > ref_end_idx:
            if start_idx > ref_end_idx and avg_Sc > 0 and current_avg/avg_Sc > threshold:
                #print("Change at "+ str(end_idx))
                changes.append(end_idx)
                #update reference
                start_idx = end_idx
                end_idx = start_idx + window_size
                reference_window, bin_edges, dim, pca , ref_hists, num_selected_components= get_reference_window(data[start_idx: end_idx], dim)
                ref_end_idx = end_idx
                current_avg = 0
                distances = []
        
        start_idx += slide_size
        end_idx += slide_size
        #lis_sequence.append(lis)
        
        
        
        
    #fig = plt.figure()
    #plt.plot(lis_sequence)
    #fig.show()
    
    return changes
    

In [6]:
truncating_number = 3
def change_detection_truncate(data, window_size, slide_size):
    threshold = 2*math.sqrt(window_size/slide_size)
    
    #threshold = window_size/slide_size*0.5
    global truncating_numbercompute_score
    global alpha
    
    dim = len(data[0,:])
    #print("dim = "+ str(dim))
    start_idx = 0
    end_idx = window_size
    #get reference 
    reference_window = data[0:window_size,:]
    #print(reference_window[4999])
    ref_hists = []
    bin_edges = []
    num_selected_components = 0
    
    
    reference_window, bin_edges, dim, pca, ref_hists,num_selected_components = get_reference_window(reference_window, dim)
    #alpha = x1/(len(bin_edges)-1)
    #print("Num bin = " + str(len(bin_edges[0])))
    
    count_slide = 0
    start_idx += slide_size
    end_idx += slide_size
    ref_end_idx = end_idx
    distances = []
    lis_sequence = []
    changes = []
    
    possible_lis = 0
    
    while(end_idx < len(data)):
        count_slide +=1
        
        current_window = data[start_idx: end_idx]
        #current_window = pca.transform(preprocessing.normalize(current_window, norm='l2'))
        current_window = pca.transform(current_window)
        #get current hist
        current_hists = get_hists(current_window,num_selected_components, bin_edges)
        #distance = distance2(current_hists,ref_hists, pca)
        distance = distance_max2(current_hists,ref_hists,pca)
        #distance = distance_min2(current_hists,ref_hists,pca)
        distance = round (Decimal(distance),truncating_number)
        #distance = distance_min(current_hists,ref_hists, pca)
        #distance = distance_sum(current_hists,ref_hists, pca)
        #distance = distance_sum_square(current_hists,ref_hists, pca)
        if count_slide %50000 == 0:
            
            print("Count slide = "+ str(count_slide))
            #print("distance = "+ str(distance))
        if start_idx > ref_end_idx:
            distances.append(distance)
        
        
        
        
        if len(distances) > window_size/slide_size:
            del distances[0]
        #print(distance)
        if start_idx > ref_end_idx:
            possible_lis +=1 
        
        if possible_lis > threshold:
            lis = len(subsequence(distances))
            possible_lis = lis
            if start_idx > ref_end_idx and lis > threshold:
                #print("Change at "+ str(end_idx))
                changes.append(end_idx)
                #update reference
                #start_idx = end_idx
                #end_idx = start_idx + window_size
                reference_window, bin_edges, dim, pca , ref_hists, num_selected_components= get_reference_window(data[start_idx: end_idx], dim)
                ref_end_idx = end_idx
                possible_lis = 0
                distances = []
        
        start_idx += slide_size
        end_idx += slide_size
        #lis_sequence.append(lis)
        
        
        
        
    #fig = plt.figure()
    #plt.plot(lis_sequence)
    #fig.show()
    
    return changes
    

In [8]:
change_types = ["sudden", "incremental", "gradual"]
change_params = [ "mean",
                 "variance"
                 , "correlation"
]
change_values = {"mean":[0.03], "variance":[0.2],
                 "correlation":[ 0.1]}
slide_sizes = [1,20,100,200,400]



truncating_number = 3
threshold_avg = 1.3
Sigma = 500
delta = 0.005
#vary change value, sudden
for change_param in change_params:
    change_type = "sudden"
    
    for slide_size in slide_sizes:
        for change_value in change_values[change_param]:
            w_precisions = {"dlis":[], "avg":[], "phdt":[]}
            w_recalls = {"dlis":[], "avg":[], "phdt":[]}
            w_f1s = {"dlis":[], "avg":[], "phdt":[]}
            for time_ in range(1):
                #create data set
                mean = [0.01, 0.01]
                cov = [[0.04, 0.02],[0.02, 0.04]]
                groud_truth = []
                d = generate_dataset("gauss",2, 50000,mean, cov, change_value, change_type, change_param, 20)
                #dlis
                changes = change_detection_truncate(d, 10000, 20)
                w_p, w_r, w_f = compute_score( groud_truth,changes,decay_factor=0.1,window_size=10000)
                if not math.isnan(w_p+w_r+w_f):
                    w_precisions["dlis"].append(w_p)
                    w_recalls["dlis"].append(w_r)
                    w_f1s["dlis"].append(w_f)
                #avg
                changes = change_detection_AVG(d, 10000, 20)
                w_p, w_r, w_f = compute_score( groud_truth,changes,decay_factor=0.1,window_size=10000)
                if not math.isnan(w_p+w_r+w_f):
                    w_precisions["avg"].append(w_p)
                    w_recalls["avg"].append(w_r)
                    w_f1s["avg"].append(w_f)
                #phdt
                changes = change_detection_PHDT(d, 10000, 20)
                w_p, w_r, w_f = compute_score( groud_truth,changes,decay_factor=0.1,window_size=10000)
                if not math.isnan(w_p+w_r+w_f):
                    w_precisions["phdt"].append(w_p)
                    w_recalls["phdt"].append(w_r)
                    w_f1s["phdt"].append(w_f)
                #print(w_precisions)
                #print(w_recalls)
                #print(w_f1s)
            #print results
            print("change "+change_param +", slide size = " +str(slide_size) + "change value = "+str(change_value))
            print("DLIS")
            print(str(np.sum(w_precisions["dlis"])/len(w_precisions["dlis"])) +"-"+
                  str(np.sum(w_recalls["dlis"])/len(w_recalls["dlis"])) +"-"+str(np.sum(w_f1s["dlis"])/len(w_f1s["dlis"])) )
            print("AVG")
            print(str(np.sum(w_precisions["avg"])/len(w_precisions["avg"])) +"-"+
                  str(np.sum(w_recalls["avg"])/len(w_recalls["avg"])) +"-"+str(np.sum(w_f1s["avg"])/len(w_f1s["avg"])) )
            print("PHDT")
            print(str(np.sum(w_precisions["phdt"])/len(w_precisions["phdt"])) +"-"+
                  str(np.sum(w_recalls["phdt"])/len(w_recalls["phdt"])) +"-"+str(np.sum(w_f1s["phdt"])/len(w_f1s["phdt"])) )


NameError: global name 'get_reference_window' is not defined

In [19]:
#process real_world dataset
groud_truth = []
selected_dims = []
import time 
def sample_a_segment(dataset, length_chunk_):
    return random.sample(list(dataset), length_chunk_)
        
def sample_dataset_no_change(filename, change_type, length_chunk_, num_chunk):
    global groud_truth
    global selected_dims
    
    dataset = np.loadtxt(filename,delimiter=",")
    
    random.seed(int(time.time()))
    no_change = []
    end_chunk_list = []
    for i in range(num_chunk):
        #length_chunk = random.randint(length_chunk_, length_chunk_*3)
        length_chunk = length_chunk_
        chunk = sample_a_segment(dataset, length_chunk)
        #change in random dim
        no_change.extend(chunk[:])
        end_chunk_list.append(len(no_change))
    
    return np.array(no_change), end_chunk_list
def add_change_to_data(no_change, end_chunk_list, change_type):
    global selected_dims
    global groud_truth
    ground_truth = end_chunk_list[0:len(end_chunk_list)-1]
    out = []
    dim = len(no_change[0])
    for i in range(end_chunk_list[0]):
        out.append(no_change[i][:])
    if change_type == "double":
        for i in range(1, len(end_chunk_list)):
            selected_dim = random.randint(0, len(no_change[0])-1)
            while(len(selected_dims) > 0 and selected_dim==selected_dims[-1]):
                selected_dim = random.randint(0, len(no_change[0])-1)
            selected_dims.append(selected_dim)
            start_idx = end_chunk_list[i-1]
            end_idx = end_chunk_list[i]
            for j in range(start_idx, end_idx):
                temp_data = []
                for k in range(dim):
                    if k!= selected_dim:
                        temp_data.append(no_change[j][k])
                    else:
                        temp_data.append(no_change[j][k] * 2)
                out.append(temp_data)
    elif change_type == "gaussian":
        for i in range(1, len(end_chunk_list)):
            selected_dim = random.randint(0, len(no_change[0])-1)
            while(len(selected_dims) > 0 and selected_dim==selected_dims[-1]):
                selected_dim = random.randint(0, len(no_change[0])-1)
            selected_dims.append(selected_dim)
            start_idx = end_chunk_list[i-1]
            end_idx = end_chunk_list[i]
            data_mean = np.mean(no_change[start_idx: end_idx][selected_dim])
            data_std = np.std(no_change[start_idx: end_idx][selected_dim])
            
            
            for j in range(start_idx, end_idx):
                temp_data = []
                for k in range(dim):
                    if k!= selected_dim:
                        temp_data.append(no_change[j][k])
                    else:
                        #add noise here
                        temp_data.append(no_change[j][k]+np.random.normal(data_mean/10, data_std/10,1) )
                out.append(temp_data)
    return np.array(out)
     
        

truncating_number = 3
threshold_avg = 1.3
Sigma = 500
delta = 0.005
all_result = {}


outlier_rate = 0.05
window = 10000
slide = 100


mean = [0.01, 0.01, 0.01, 0.01, 0.01]
cov = [[0.04, 0.02, 0.02, 0.02, 0.02],[0.02, 0.04, 0.02, 0.02, 0.02],[0.02, 0.02, 0.04, 0.02, 0.02],[0.02, 0.02, 0.02, 0.04, 0.02],[0.02, 0.02, 0.02, 0.02, 0.04]]

#vary change value, sudden
for data_name in ["covtype.data", "ethylene.txt", "tao.txt", "household.txt"]:
    for error_type in ["double"]:
        print("----------------------------------------------")
        print("Data name = "+ str(data_name))
        print("Error type = "+ str(error_type))

        w_precisions = {"dlis":[], "avg":[], "phdt":[]}
        w_recalls = {"dlis":[], "avg":[], "phdt":[]}
        w_f1s = {"dlis":[], "avg":[], "phdt":[]}
        for time_ in range(1):
            #create data set
            #d = generate_dataset("gauss",2, 50000,mean, cov, change_value, change_type, change_param, 20)
            no_change, groud_truth = sample_dataset_no_change(data_name, error_type, 50000, 5)
            d = add_change_to_data(no_change, groud_truth, error_type)
            if data_name == "covtype.data":
                d = d[:, :10]
            print(groud_truth)
            #dlis
            changes = change_detection_truncate(d, 10000, 20)
            print(changes)
            w_p, w_r, w_f = compute_score( groud_truth,changes,decay_factor=0.1,window_size=10000)
            if not math.isnan(w_p+w_r+w_f):
                w_precisions["dlis"].append(w_p)
                w_recalls["dlis"].append(w_r)
                w_f1s["dlis"].append(w_f)
            #avg
            changes = change_detection_AVG(d, 10000, 20)
            w_p, w_r, w_f = compute_score( groud_truth,changes,decay_factor=0.1,window_size=10000)
            if not math.isnan(w_p+w_r+w_f):
                w_precisions["avg"].append(w_p)
                w_recalls["avg"].append(w_r)
                w_f1s["avg"].append(w_f)
            #phdt
            changes = change_detection_PHDT(d, 10000, 20)
            w_p, w_r, w_f = compute_score( groud_truth,changes,decay_factor=0.1,window_size=10000)
            if not math.isnan(w_p+w_r+w_f):
                w_precisions["phdt"].append(w_p)
                w_recalls["phdt"].append(w_r)
                w_f1s["phdt"].append(w_f)
            #print(w_precisions)
            #print(w_recalls)
            #print(w_f1s)
        #print results
        #print("change "+change_param +", slide size = " +str(slide_size) + "change value = "+str(change_value))
        print("Error type = "+ str(error_type))
        print("DLIS")
        print(str(np.sum(w_precisions["dlis"])/len(w_precisions["dlis"])) +"-"+
              str(np.sum(w_recalls["dlis"])/len(w_recalls["dlis"])) +"-"+str(np.sum(w_f1s["dlis"])/len(w_f1s["dlis"])) )
        print("AVG")
        print(str(np.sum(w_precisions["avg"])/len(w_precisions["avg"])) +"-"+
              str(np.sum(w_recalls["avg"])/len(w_recalls["avg"])) +"-"+str(np.sum(w_f1s["avg"])/len(w_f1s["avg"])) )
        print("PHDT")
        print(str(np.sum(w_precisions["phdt"])/len(w_precisions["phdt"])) +"-"+
              str(np.sum(w_recalls["phdt"])/len(w_recalls["phdt"])) +"-"+str(np.sum(w_f1s["phdt"])/len(w_f1s["phdt"])) )


----------------------------------------------
Data name = covtype.data
Error type = double
[50000, 100000, 150000, 200000, 250000]
[]


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in double_scalars
  


Error type = double
DLIS
nan-nan-nan
AVG
nan-nan-nan
PHDT
nan-nan-nan
----------------------------------------------
Data name = ethylene.txt
Error type = double


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:138: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:141: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:144: RuntimeWarning: invalid value encountered in double_scalars


KeyboardInterrupt: 

In [9]:
#process real_world dataset
groud_truth = []
def sample_dataset(filename, change_type, length_chunk, num_chunk):
    global groud_truth
    dataset = np.loadtxt(filename,delimiter=",")
    #remove nan
        #sample
    out = []
    for i in range(num_chunk):
        last_dim = -1
        chunk = []
        for j in range(length_chunk):
            idx = random.randint(0, len(dataset)-1)
            data = dataset[idx]
            chunk.append(data)
        #select dim to change
        selected_dim = random.randint(0, len(chunk[0])-1)
        while(selected_dim == last_dim):
            selected_dim = random.randint(0, len(chunk[0])-1)
        last_dim = selected_dim
        #change mean
        if change_type == "double":
            for j in range(length_chunk):
                chunk[j][selected_dim] *= 2
        #append to out
        elif change_type == "gauss":
            chunk_data =np.array(chunk)
            mean_chunk = np.mean(chunk_data[:,selected_dim])
            noise_mean = mean_chunk * 1/10
            noise_std = np.std(chunk_data[:,selected_dim]) *1/10
            noise_chunk = np.random.normal(noise_mean, noise_std, length_chunk)
            for j in range(length_chunk):
                chunk[j][selected_dim] += noise_chunk[j]
        if i > 0:
            groud_truth.append(len(out))
        for data in chunk:
            out.append(data)
        
    print(len(out))
    print(groud_truth)
    
    return np.array(out)
        
d= sample_dataset("tao.txt", "gauss", 50000, 100)

5000000
[50000, 100000, 150000, 200000, 250000, 300000, 350000, 400000, 450000, 500000, 550000, 600000, 650000, 700000, 750000, 800000, 850000, 900000, 950000, 1000000, 1050000, 1100000, 1150000, 1200000, 1250000, 1300000, 1350000, 1400000, 1450000, 1500000, 1550000, 1600000, 1650000, 1700000, 1750000, 1800000, 1850000, 1900000, 1950000, 2000000, 2050000, 2100000, 2150000, 2200000, 2250000, 2300000, 2350000, 2400000, 2450000, 2500000, 2550000, 2600000, 2650000, 2700000, 2750000, 2800000, 2850000, 2900000, 2950000, 3000000, 3050000, 3100000, 3150000, 3200000, 3250000, 3300000, 3350000, 3400000, 3450000, 3500000, 3550000, 3600000, 3650000, 3700000, 3750000, 3800000, 3850000, 3900000, 3950000, 4000000, 4050000, 4100000, 4150000, 4200000, 4250000, 4300000, 4350000, 4400000, 4450000, 4500000, 4550000, 4600000, 4650000, 4700000, 4750000, 4800000, 4850000, 4900000, 4950000]


In [10]:
#generate high dimensional data
dim = 5
groud_truth = []

mean = []
cov = []
for i in range(dim):
    mean.append(0.5)
    cov.append([])
for i in range(len(cov)):
    cov[i]= []
    for j in range(dim):
        cov[i].append(0)
    for j in range(len(cov[i])):
        if j == i:
            cov[i][j] = 0.04
        else:
            cov[i][j] = 0.02
num_change_dimension = 1

def generate_dataset_high_dim(distribution_type, dim, length_chunk, mean, cov, change_value, change_type,change_param, num_chunk):
    global num_change_dimension
    #change mean
    out = []
    ground_truth = []
    if distribution_type == "gauss":
        for i in range(num_chunk):
            generated_length_chunk = random.randint(length_chunk,length_chunk * 4)
            x = np.random.multivariate_normal(mean, cov, generated_length_chunk)
            if change_param == "mean":
                if change_type == "sudden":
                    for j  in range(num_change_dimension):
                        mean[j] += change_value
            
            elif change_param == "correlation":
                if change_type == "sudden":
                    k = random.randint(0, dim-1)
                    j = random.randint(0, dim-1)
                    while j == k:
                        j = random.randint(0, dim-1)
                    if j != k:

                        if cov[j][k]+change_value * cov[j][j] < cov[j][j]:
                            cov[j][k] += change_value * cov[j][j]
                            cov[k][j] = cov[j][k]
                        else :
                            cov[j][k] = 0.5 * cov[j][j]
                            cov[k][j] = cov[j][k]    
                    
                    print(cov)
            if i > 0:
                groud_truth.append(len(out))
            if change_type == "sudden" or len(out) == 0:
                for xi in x:
                    out.append(xi)
        
    return np.array(out)

In [7]:
def subsequence3(seq):
    if not seq:
        return seq

    M = [None] * len(seq)    # offset by 1 (j -> j-1)
    P = [None] * len(seq)

    # Since we have at least one element in our list, we can start by
    # knowing that the there's at least an increasing subsequence of length one:
    # the first element.
    L = 1
    M[0] = 0

    # Looping over the sequence starting from the second element
    for i in range(1, len(seq)):
        # Binary search: we want the largest j <= L
        #  such that seq[M[j]] < seq[i] (default j = 0),
        #  hence we want the lower bound at the end of the search process.
        lower = 0
        upper = L

        # Since the binary search will not look at the upper bound value,
        # we'll have to check that manually
        if seq[M[upper-1]] < seq[i] and (P[M[upper-1]] == None or seq[i] > 2*seq[M[upper-1]] - seq[P[M[upper-1]]])  :
            j = upper

        else:
            # actual binary search loop
            while upper - lower > 1:
                mid = (upper + lower) // 2
                if seq[M[mid-1]] < seq[i] and (P[M[mid-1]] == None or 2*seq[M[mid-1]] - seq[P[M[mid-1]]]< seq[i]):
                    lower = mid
                else:
                    upper = mid

            j = lower    # this will also set the default value to 0

        P[i] = M[j-1]

        if j == L or seq[i] < seq[M[j]]:
            M[j] = i
            L = max(L, j+1)

    # Building the result: [seq[M[L-1]], seq[P[M[L-1]]], seq[P[P[M[L-1]]]], ...]
    result = []
    pos = M[L-1]
    for _ in range(L):
        result.append(seq[pos])
        pos = P[pos]

    return result[::-1]    # reversing

In [8]:
def get_reference_window(curren_window, dim):
    ref_hists = []
    bin_edges = []
    num_selected_components = 0
    
    pca = PCA(n_components=dim)
    #normalized_current_window = preprocessing.normalize(curren_window, norm='l2')
    pca.fit(curren_window)
    
    s = 0
    
    for i in range(len(pca.explained_variance_ratio_)):
        s+= pca.explained_variance_ratio_[i]
        if s >= 0.999:
            num_selected_components = i + 1
            break
    #print("Num selected components = "+ str(num_selected_components))
    reference_window = pca.transform(curren_window)
    for i in range(num_selected_components):
        hist, bin_edge = np.histogram(reference_window[:,i], 'auto')
        for j in range(len(hist)):
            hist[j] += 1
        ref_hists.append(hist)
        bin_edges.append(bin_edge)
    return reference_window, bin_edges, dim, pca, ref_hists, num_selected_components
def get_hists(current_window, dim, bin_edges):
    current_hists = []
    for i in range(dim):
        hist, edge = np.histogram(current_window[:,i], bin_edges[i])
        for j in range(len(hist)):
            hist[j] += 1
        current_hists.append(hist)
    return current_hists
def distance1(hist1, hist2):
    dim = len(hist1)
    dlist=  []
    for i in range(dim):
        d1 = mkl(hist1[i], hist2[i])
        #d2 = kl(hist2[i], hist1[i])
        #print("d = "+str(d1))
        #print("d2 = "+str(d2))
        dlist.append(d1)

    return np.max(dlist)
def distance2(hist1, hist2, pca):
    dim = len(hist1)
    dlist=  []
    for i in range(dim):
        d1 = kl(hist1[i], hist2[i])
        #d2 = kl(hist2[i], hist1[i])
        #print("d = "+str(d1))W_precision = 0.9
        #print("d2 = "+str(d2))
        dlist.append(d1)

    return np.sum([dlist[i]*pca.explained_variance_ratio_[i] for i in range(len(dlist))])
def distance_max(hist1, hist2, pca):
    dim = len(hist1)
    dlist=  []
    for i in range(dim):
        d1 = kl(hist1[i], hist2[i])
        #d2 = kl(hist2[i], hist1[i])
        #print("d = "+str(d1))W_precision = 0.9
        #print("d2 = "+str(d2))
        dlist.append(d1)

    return np.max(dlist)
def distance_max2(hist1, hist2, pca):
    dim = len(hist1)
    dlist=  []
    for i in range(dim):
        d1 = kl2(hist1[i], hist2[i])
        #d2 = kl(hist2[i], hist1[i])
        #print("d = "+str(d1))W_precision = 0.9
        #print("d2 = "+str(d2))
        dlist.append(d1)

    return np.max(dlist)
def distance_sum(hist1, hist2, pca):
    dim = len(hist1)
    dlist=  []
    for i in range(dim):
        d1 = kl(hist1[i], hist2[i])
        #d2 = kl(hist2[i], hist1[i])
        #print("d = "+str(d1))W_precision = 0.9
        #print("d2 = "+str(d2))
        dlist.append(d1)

    return np.sum(dlist)
def distance_sum_square(hist1, hist2, pca):
    dim = len(hist1)
    dlist=  []
    for i in range(dim):
        d1 = kl(hist1[i], hist2[i])
        #d2 = kl(hist2[i], hist1[i])
        #print("d = "+str(d1))W_precision = 0.9
        #print("d2 = "+str(d2))
        dlist.append(d1)

    return np.sum([x*x for x in dlist])
def distance_min(hist1, hist2, pca):
    dim = len(hist1)
    dlist=  []
    for i in range(dim):
        d1 = kl(hist1[i], hist2[i])
        #d2 = kl(hist2[i], hist1[i])
        #print("d = "+str(d1))W_precision = 0.9
        #print("d2 = "+str(d2))
        dlist.append(d1)

    return np.min(dlist)

def distance_min2(hist1, hist2, pca):
    dim = len(hist1)
    dlist=  []
    for i in range(dim):
        d1 = kl2(hist1[i], hist2[i])
        #d2 = kl(hist2[i], hist1[i])
        #print("d = "+str(d1))W_precision = 0.9
        #print("d2 = "+str(d2))
        dlist.append(d1)

    return np.min(dlist)

In [9]:
def subsequence(seq):
    if not seq:
        return seq

    M = [None] * len(seq)    # offset by 1 (j -> j-1)
    P = [None] * len(seq)

    # Since we have at least one element in our list, we can start by 
    # knowing that the there's at least an increasing subsequence of length one:
    # the first element.
    L = 1
    M[0] = 0

    # Looping over the sequence starting from the second element
    for i in range(1, len(seq)):
        # Binary search: we want the largest j <= L
        #  such that seq[M[j]] < seq[i] (default j = 0),
        #  hence we want the lower bound at the end of the search process.
        lower = 0
        upper = L

        # Since the binary search will not look at the upper bound value,
        # we'll have to check that manually
        if seq[M[upper-1]] < seq[i]:
            j = upper

        else:
            # actual binary search loop
            while upper - lower > 1:
                mid = (upper + lower) // 2
                if seq[M[mid-1]] < seq[i]:
                    lower = mid
                else:
                    upper = mid

            j = lower    # this will also set the default value to 0

        P[i] = M[j-1]

        if j == L or seq[i] < seq[M[j]]:
            M[j] = i
            L = max(L, j+1)

    # Building the result: [seq[M[L-1]], seq[P[M[L-1]]], seq[P[P[M[L-1]]]], ...]
    result = []
    pos = M[L-1]
    for _ in range(L):
        result.append(seq[pos])
        pos = P[pos]

    return result[::-1]    # reversing
    

In [10]:
alpha = 0.00001
def subsequence2(seq):
    global alpha
    if not seq:
        return seq

    M = [None] * len(seq)    # offset by 1 (j -> j-1)
    P = [None] * len(seq)

    # Since we have at least one element in our list, we can start by 
    # knowing that the there's at least an increasing subsequence of length one:
    # the first element.
    L = 1
    M[0] = 0

    # Looping over the sequence starting from the second element
    for i in range(1, len(seq)):
        # Binary search: we want the largest j <= L
        #  such that seq[M[j]] < seq[i] (default j = 0),
        #  hence we want the lower bound at the end of the search process.
        lower = 0
        upper = L

        # Since the binary search will not look at the upper bound value,
        # we'll have to check that manually
        if seq[M[upper-1]] + alpha < seq[i]:
            j = upper

        else:
            # actual binary search loop
            while upper - lower > 1:
                mid = (upper + lower) // 2
                if seq[M[mid-1]] + alpha < seq[i]:
                    lower = mid
                else:
                    upper = mid

            j = lower    # this will also set the default value to 0

        P[i] = M[j-1]

        if j == L or seq[i] + alpha < seq[M[j]]:
            M[j] = i
            L = max(L, j+1)

    # Building the result: [seq[M[L-1]], seq[P[M[L-1]]], seq[P[P[M[L-1]]]], ...]
    result = []
    pos = M[L-1]
    for _ in range(L):
        result.append(seq[pos])
        pos = P[pos]

    return result[::-1]    # reversing
    

In [13]:
print(subsequence2([1, 4, 2, 5, 7, 1, 0]))

[1, 2, 5, 7]


In [11]:
x1 = 0.00043
alpha = 0.0001

def change_detection(data, window_size, slide_size):
    threshold = 2*math.sqrt(window_size/slide_size)
    
    global alpha
    
    dim = len(data[0,:])
    #print("dim = "+ str(dim))
    start_idx = 0
    end_idx = window_size
    #get reference 
    reference_window = data[0:window_size,:]
    #print(reference_window[4999])
    ref_hists = []
    bin_edges = []
    num_selected_components = 0
    
    
    reference_window, bin_edges, dim, pca, ref_hists,num_selected_components = get_reference_window(reference_window, dim)
    #alpha = x1/(len(bin_edges)-1)
    print("Num bin = " + str(len(bin_edges[0])))
    
    count_slide = 0
    start_idx += slide_size
    end_idx += slide_size
    ref_end_idx = end_idx
    distances = []
    lis_sequence = []
    changes = []
    
    possible_lis = 0
    
    while(end_idx < len(data)):
        count_slide +=1
        
        current_window = data[start_idx: end_idx]
        current_window = pca.transform(current_window)
        #get current hist
        current_hists = get_hists(current_window,num_selected_components, bin_edges)
        #distance = distance2(current_hists,ref_hists, pca)
        distance = distance_max(ref_hists,current_hists, pca)
        #distance = distance_min(current_hists,ref_hists, pca)
        #distance = distance_sum(current_hists,ref_hists, pca)
        #distance = distance_sum_square(current_hists,ref_hists, pca)
        if count_slide %50000 == 0:
            print("Count slide = "+ str(count_slide))
            #print("distance = "+ str(distance))
        if start_idx > ref_end_idx:
            distances.append(distance)
        
        
        
        
        if len(distances) > window_size/slide_size:
            del distances[0]
        #print(distance)
        if start_idx > ref_end_idx:
            possible_lis +=1 
        
        if possible_lis > threshold:
            lis = len(subsequence(distances))
            possible_lis = lis
            if start_idx > ref_end_idx and lis > threshold:
                #print("Change at "+ str(end_idx))
                changes.append(end_idx)
                #update reference
                reference_window, bin_edges, dim, pca , ref_hists, num_selected_components= get_reference_window(data[start_idx: end_idx], dim)
                ref_end_idx = end_idx
                possible_lis = 0
                distances = []
        
        start_idx += slide_size
        end_idx += slide_size
        #lis_sequence.append(lis)
        
        
        
        
    #fig = plt.figure()
    #plt.plot(lis_sequence)
    #fig.show()
    
    return changes
    

In [12]:
step_from_beginning = 0
m_current = 0
M_current = 0
averaged_sc = 0
def change_PHDT(curScore, Sigma, delta):
    global step_from_beginning
    global m_current
    global M_current
    global averaged_sc
    averaged_sc = (averaged_sc * step_from_beginning + curScore)*1.0/(step_from_beginning+1)
    m_current = m_current + averaged_sc  - curScore + delta
    if abs(m_current) > M_current:
        M_current = abs(m_current)
    theta_t = Sigma * averaged_sc
    if (M_current - m_current) > theta_t:
        return True
    
    step_from_beginning += 1
    return False

In [11]:
Sigma = 500
delta = 0.0005
def change_detection_PHDT(data, window_size, slide_size):
    global step_from_beginning
    global averaged_sc
    global m_current 
    global M_current
    
    
    changes = []
    global Sigma
    global delta
    dim = len(data[0,:])
    num_selected_components = 0
    #print("dim = "+ str(dim))
    start_idx = 0
    end_idx = window_size
    ref_end_idx = end_idx
    #get reference 
    reference_window = data[0:window_size,:]
    #print(reference_window[4999])
    
    step_from_beginning = 0
    averaged_sc = 0
    
    reference_window, bin_edges, dim, pca, ref_hists,num_selected_components = get_reference_window(reference_window, dim)
    
    count_slide = 0
    start_idx += slide_size
    end_idx += slide_size
    distances = []
    
    
    while(end_idx < len(data)):
        count_slide +=1
        
            
        current_window = data[start_idx: end_idx]
        current_window = pca.transform(current_window)
        #get current hist
        current_hists = get_hists(current_window,num_selected_components, bin_edges)
        distance = distance1(current_hists,ref_hists)
        distances.append(distance)
        
        if count_slide %50000 == 0:
            #print("Count slide = "+ str(count_slide))
            #print("averaged_sc = "+ str(averaged_sc))
            #print("Distance = " + str(distance))
            #print("M = "+str(M_current))
            #print("m = "+ str(m_current))
            pass
        if(start_idx > ref_end_idx and  change_PHDT(distance, Sigma, delta)):
            #print("Change at : "+ str(end_idx))
            
            averaged_sc = 0
            M_current = 0
            m_current = 0
            step_from_beginning = 0
            changes.append(end_idx)
            start_idx = end_idx
            end_idx = start_idx + window_size
            #update reference
            reference_window, bin_edges, dim, pca , ref_hists, num_selected_components= get_reference_window(data[start_idx: end_idx], dim)
            ref_end_idx = end_idx
        end_idx += slide_size
        start_idx += slide_size
    return changes

In [13]:
def compute_score(ground_truth, result, decay_factor, window_size):
    scores = [0 for x in result]
    for i in range(len(result)):
        #check if result[i] is the soonest detected change point in a distribution 
        rs = result[i]
        #----find the closest left ground_truth
        left = -1
        for j in range(len(ground_truth)):
            if ground_truth[j] <= rs and (i == 0 or (i > 0 and result[i-1] < ground_truth[j])) :
                left = ground_truth[j]
        if left != -1:
            scores[i] =  np.exp(-decay_factor*(int((rs - left)/window_size)))
    #print(scores)
    w_precision = np.sum(scores)/len(result)
    w_recall = np.sum(scores)/len(groud_truth)
    w_f1 = 2*w_precision*w_recall/(w_precision+w_recall)
    #print("W_precision = "+str(w_precision))
    #print("W_Recall = "+ str(w_recall))
    #print("W_f1 = "+str(w_f1))
    return w_precision, w_recall, w_f1
        
    

In [14]:
def compute_f1_score(groudtruths, all_outliers):
    scores = []
    for i in range(len(groudtruths)):
        ground = groudtruths[i]
        outliers = all_outliers[i]
        #if (len(ground) != len(outliers)):
            #print("Different length!!!!!!")
        score = 0
        for k in range(len(outliers)):
            if outliers[k] in ground:
                score +=1
        precision, recall = 0,0
        if len(outliers) > 0:
            precision = score *1.0/len(outliers)
        recall = score *1.0/len(ground)
        if precision + recall > 0:
            f1 = 2* precision * recall / (precision + recall)
            scores.append(f1)
        else:
            scores.append(0)
    return np.sum(scores)/len(scores)



In [21]:
#generate high dimensional data
dim = 5
groud_truth = []

mean = []
cov = []
for i in range(dim):
    mean.append(0.5)
    cov.append([])
for i in range(len(cov)):
    cov[i]= []
    for j in range(dim):
        cov[i].append(0)
    for j in range(len(cov[i])):
        if j == i:
            cov[i][j] = 0.04
        else:
            cov[i][j] = 0.02
num_change_dimension = 1

def generate_dataset_high_dim(distribution_type, dim, length_chunk, mean, cov, change_value, change_type,change_param, num_chunk):
    global num_change_dimension
    #change mean
    out = []
    ground_truth = []
    if distribution_type == "gauss":
        for i in range(num_chunk):
            generated_length_chunk = random.randint(length_chunk,length_chunk * 10)
            x = np.random.multivariate_normal(mean, cov, generated_length_chunk)
            if change_param == "mean":
                if change_type == "sudden":
                    for j  in range(num_change_dimension):
                        mean[j] += change_value
            
            elif change_param == "correlation":
                if change_type == "sudden":
                    k = random.randint(0, dim-1)
                    j = random.randint(0, dim-1)
                    while j == k:
                        j = random.randint(0, dim-1)
                    if j != k:

                        if cov[j][k]+change_value * cov[j][j] < cov[j][j]:
                            cov[j][k] += change_value * cov[j][j]
                            cov[k][j] = cov[j][k]
                        else :
                            cov[j][k] = 0.5 * cov[j][j]
                            cov[k][j] = cov[j][k]    
                    
                    print(cov)
            if i > 0:
                groud_truth.append(len(out))
            if change_type == "sudden" or len(out) == 0:
                for xi in x:
                    out.append(xi)
        
    return np.array(out)